<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#词向量的概念" data-toc-modified-id="词向量的概念-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>词向量的概念</a></span><ul class="toc-item"><li><span><a href="#one-hot编码存在的问题" data-toc-modified-id="one-hot编码存在的问题-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>one-hot编码存在的问题</a></span></li><li><span><a href="#Word2Vec" data-toc-modified-id="Word2Vec-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Word2Vec</a></span></li></ul></li><li><span><a href="#词向量的实现方法" data-toc-modified-id="词向量的实现方法-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>词向量的实现方法</a></span><ul class="toc-item"><li><span><a href="#CBOW" data-toc-modified-id="CBOW-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>CBOW</a></span></li><li><span><a href="#Skip-Gram" data-toc-modified-id="Skip-Gram-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Skip-Gram</a></span></li></ul></li><li><span><a href="#TensorFlow实现词向量" data-toc-modified-id="TensorFlow实现词向量-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>TensorFlow实现词向量</a></span><ul class="toc-item"><li><span><a href="#载入库文件" data-toc-modified-id="载入库文件-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>载入库文件</a></span></li><li><span><a href="#收集数据" data-toc-modified-id="收集数据-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>收集数据</a></span><ul class="toc-item"><li><span><a href="#下载数据" data-toc-modified-id="下载数据-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>下载数据</a></span></li><li><span><a href="#解压数据" data-toc-modified-id="解压数据-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>解压数据</a></span></li></ul></li></ul></li></ul></div>

# 词向量的概念

## one-hot编码存在的问题

自然语言处理在Word2Vec出现之前，通常将字词转成离散的单独的符号，比如将“中国”转为编号为5178的特征，将“北京”转为编号为3987的特征。这即是One-Hot Encoder，一个词对应一个向量（向量中只有一个值为1，其余为0），通常需要将一篇文章中每一个词都转成一个向量，而整篇文章则变为一个稀疏矩阵。

对文本分类模型，我们使用Bag of Words模型，将文章对应的稀疏矩阵合并为一个向量，即把每一个词对应的向量加到一起，这样只统计每个词出现的次数，比如“中国”出现23次，那么第5178个特征为23，“北京”出现2次，那么第3987个特征为2。

使用One-Hot Encoder存在以下两个问题

1. 特征编码是随机的, 没有考虑字词间存在的关系.例如，我们对“中国”和“北京”的从属关系、地理位置关系等一无所知，我们从5178和3987这两个值看不出任何信息。
2. 训练效率低, 计算麻烦.将字词存储为稀疏向量的话，我们通常需要更多的数据来训练，因为稀疏数据训练的效率比较低，计算也非常麻烦.

## Word2Vec

Word2Vec则是将语言中的字词转化为计算机可以理解的稠密向量（DenseVector）.

Word2Vec 会将每一个单词表示成一个相对较低维度的向量(比如100维或者200维)。对于语义相近的词，其对应的单词向量在空间中的距离也应该接近。于是单词语义上的相似度可以通过空间距离来描述。单词向量不需要通过人工的方式设定，它可以从互联网上海量非标注文本中学习得到。使用斯坦福大学开源的 GloVe 单词向量可以得到与单词 "frog" (青蛙)所对应的单词向量最相似的5个单词分别是 "frogs (青蛙复数)"，"toad(蟾蜍)"，“litoria(雨滨蛙属)”，“leptodactylidae (细趾蟾科)”和 “rana (中国林蛙)”。从这个样例可以看出，单词向量可以非常有效的刻画单词的语义。通过单词向量还可以进行单词之间的运算。比如用单词“king”所代表的向量去减去"man"所代表的向量得到的结果和单词 "queen" 减去 "woman" 得到的结果向量是相似的。这说明在单词向量中，已经隐含了表达性别的概念.

使用Word2Vec学习到的字词间对应关系如下:

![6_1](6_1.jpg)

# 词向量的实现方法

Word2Vec 即是一种计算非常高效的，可以从原始语料中学习字词空间向量的预测模型。它主要分为CBOW（ContinuousBag of Words）和 Skip-Gram 两种模式，其中CBOW是从原始语句（比如：中国的首都是____）推测目标字词（比如：北京）；而 Skip-Gram 则正好相反，它是从目标字词推测出原始语句. 其中CBOW对小型数据比较合适，而Skip-Gram在大型语料中表现得更好.

## CBOW

预测模型 Neural Probabilistic Language Models 通常使用最大似然的方法，在给定前面的语句 h 的情况下，最大化目标词汇 wt 的概率。但它存在的一个比较严重的问题是计算量非常大，需要计算词汇表中所有单词出现的可能性。在 Word2Vec 的CBOW模型中，不需要计算完整的概率模型，只需要训练一个二元的分类模型，用来区分真实的目标词汇和编造的词汇（噪声）这两类。这种用少量噪声词汇来估计的方法，类似于蒙特卡洛模拟。CBOW 模型如下所示:

![6_2](6_2.png)

当模型预测真实的目标词汇为高概率，同时预测其他噪声词汇为低概率时，我们训练的学习目标就被最优化了。用编造的噪声词汇训练的方法被称为 Negative Sampling。用这种方法计算loss function的效率非常高，我们只需要计算随机选择的 k 个词汇而非词汇表中的全部词汇，因此训练速度非常快。在实际中，我们使用 Noise-Contrastive Estimation（NCE） Loss，同时在TensorFlow 中也有 tf.nn.nce_loss() 直接实现了这个 loss 。

<font color=red>对 NCE 的理解还是有点不明白 </font>

## Skip-Gram


**Skip-Gram的样本构造**


以 “the quick brown fox jumpedover the lazy dog” 这句话为例。我们要构造一个语境与目标词汇的映射关系，其中语境包括一个单词左边和右边的词汇，假设我们的滑窗尺寸为1，可以制造的映射关系包括［the, brown］→quick、［quick, fox］→brown、［brown, jumped］→fox等.因为Skip-Gram模型是从目标词汇预测语境，所以训练样本不再是［the, brown］→quick，而是quick→the和quick→brown。我们的数据集就变为了（quick, the）、（quick, brown）、（brown,quick）、（brown, fox）等.

我们训练时，希望模型能从目标词汇quick预测出语境the，同时也需要制造随机的词汇作为负样本（噪声），我们希望预测的概率分布在正样本the上尽可能大，而在随机产生的负样本上尽可能小。这里的做法就是通过优化算法比如 SGD 来更新模型中 Word Embedding 的参数，让概率分布的损失函数（NCE Loss）尽可能小。这样每个单词的 EmbeddedVector 就会随着训练过程不断调整，直到处于一个最适合语料的空间位置。这样我们的损失函数最小，最符合语料，同时预测出正确单词的概率也最高.

# TensorFlow实现词向量

本节用 Skip-Gram 的方式实现 Word2Vec.

## 载入库文件

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import sys
import argparse
import random
from tempfile import gettempdir
import zipfile

import numpy as np
from six.moves import urllib
from six.moves import xrange # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.tensorboard.plugins import projector

## 收集数据

### 下载数据

我们先定义下载文本数据的函数。这里使用urllib.re-quest.urlretrieve下载数据的压缩文件并核对文件尺寸，如果已经下载了文件则跳过。

In [4]:
# step1: Download the data
url = 'http://mattmahoney.net/dc/'

# 定义下载数据的函数并对数据大小进行校准
def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    # local_filename = os.path.join(gettempdir(), filename)
    local_filename = os.path.join("../../../TensorFlow/datasets/", filename)
    
    # 如果文件不存在, 则下载文件
    if not os.path.exists(local_filename):
        local_filename, _ = urllib.request.urlretrieve(url + filename, local_filename)
    
    # 获取下载文件的信息, 并进行大小验证
    statinfo = os.stat(local_filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified', filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + local_filename + '. Can you get to it with a browser?')
    
    return local_filename

# 下载文件, 并进行大小验证 
filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


### 解压数据

接下来解压下载的压缩文件，并使用`tf.compat.as_str`将数据转成单词的列表。通过程序输出，可以知道数据最后被转为了一个包含 $17005207$个单词的列表。

打印前50个单词,可以发现和文本是一一对应的.

In [6]:
# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    
  return data

# 读取文件并把文件转化为单词列表
vocabulary = read_data(filename)
print('Data size', len(vocabulary))
print('Data [:50] ', vocabulary[:50])

Data size 17005207
Data [:50]  ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the']


In [ ]:
# 参考资料